# Calculating Forest Loss at the Ethnologue Polygon Level

In [ ]:
import os
import ee
import geemap 
import pandas as pd

#ee.Authenticate()
ee.Initialize()

In [ ]:
# Define region to view (Peru/Brazil Amazon)
world_bbox = ee.Geometry.BBox(-180, -85, 180, 85)

# Load the treecover2000 band
treecover = ee.Image("UMD/hansen/global_forest_change_2024_v1_12").select("treecover2000")

# Visualization settings
vis_params = {
    'min': 0,
    'max': 100,
    'palette': ['ffffff', '006400']
}

# Create map
Map = geemap.Map(center=[-7.5, -72.5], zoom=6)
Map.addLayer(treecover.clip(region), vis_params, 'Treecover 2000')
Map.addLayer(world_bbox, {}, 'Region')
Map

In [ ]:
# Export to Google Drive
task = ee.batch.Export.image.toDrive(
    image=treecover,
    description='treecover2000_100m_30m',
    folder='GEE_exports',
    fileNamePrefix='treecover2000_100m_30m',
    region=world_bbox,
    scale=100,
    maxPixels=1e13
)

task.start()

In [ ]:
# Load the treecover2000 band
treeloss = ee.Image("UMD/hansen/global_forest_change_2024_v1_12").select("loss")

# Export to Google Drive
task = ee.batch.Export.image.toDrive(
    image=treeloss,
    description='treeloss_100m_30m',
    folder='GEE_exports',
    fileNamePrefix='treeloss_100m_30m',
    region=world_bbox,
    scale=100,
    maxPixels=1e13
)

task.start()

### NOTE: see the progress of exports at [GEE Task Manager](https://code.earthengine.google.com/tasks)

In [35]:
import os
from pathlib import Path

import pandas as pd
import numpy as np
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
import matplotlib.patches as mpatches
from matplotlib.font_manager import FontProperties

import mapclassify

from rapidfuzz import process, fuzz

from shapely.geometry import Point

import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from rasterstats import zonal_stats
from glob import glob
from rasterio.merge import merge
from rasterio.enums import Resampling

In [33]:
# Set base project path
base_path = Path("C:/Users/juami/Dropbox/RAships/2-Folklore-Nathan-Project/EA-Maps-Nathan-project/Measures_work")

# Set file paths
poscol_path = base_path / "data" / "raw" / "ethnologue" / "ancestral_characteristics_database_language_level" / "Ethnologue_16_shapefile" / "langa_no_overlap_biggest_clean.shp"

data_path = base_path / "data" / "interim"
maps_path = base_path / "maps" / "raw"
hansen_path = base_path / "maps" / "raw" / "Hansen_forest"

export_path = base_path / "maps" / "interim" / "ethnologue_parts"

In [28]:
# Read the shapefiles and CSV files
ethnologue = gpd.read_file(poscol_path)

# Keep only relevant columns
ethnologue = ethnologue[["ID", "geometry"]]

print(f"Number of features: {len(ethnologue)}")

Number of features: 7087


In [ ]:
# List all the raster files (assuming they are .tif files)
raster_files = glob(str(hansen_path / "*.tif"))

print("Found raster files:", raster_files)

# Read in all the rasters
src_files_to_mosaic = [rasterio.open(fp) for fp in raster_files]

from rasterio.merge import merge

# Set a coarser resolution
mosaic, out_trans = merge(
    src_files_to_mosaic,
    res=(0.01, 0.01),  # Approx ~1km resolution (in degrees)
    resampling=Resampling.average  # Can also try Resampling.nearest
)

Found raster files: ['C:\\Users\\juami\\Dropbox\\RAships\\2-Folklore-Nathan-Project\\EA-Maps-Nathan-project\\Measures_work\\maps\\raw\\Hansen_forest\\treecover2000_100m_30m-0000000000-0000000000.tif', 'C:\\Users\\juami\\Dropbox\\RAships\\2-Folklore-Nathan-Project\\EA-Maps-Nathan-project\\Measures_work\\maps\\raw\\Hansen_forest\\treecover2000_100m_30m-0000000000-0000065536.tif', 'C:\\Users\\juami\\Dropbox\\RAships\\2-Folklore-Nathan-Project\\EA-Maps-Nathan-project\\Measures_work\\maps\\raw\\Hansen_forest\\treecover2000_100m_30m-0000000000-0000131072.tif', 'C:\\Users\\juami\\Dropbox\\RAships\\2-Folklore-Nathan-Project\\EA-Maps-Nathan-project\\Measures_work\\maps\\raw\\Hansen_forest\\treecover2000_100m_30m-0000000000-0000196608.tif', 'C:\\Users\\juami\\Dropbox\\RAships\\2-Folklore-Nathan-Project\\EA-Maps-Nathan-project\\Measures_work\\maps\\raw\\Hansen_forest\\treecover2000_100m_30m-0000000000-0000262144.tif', 'C:\\Users\\juami\\Dropbox\\RAships\\2-Folklore-Nathan-Project\\EA-Maps-Nathan-

In [ ]:
# Show the first band of the mosaic directly
fig, ax = plt.subplots(figsize=(10, 10))
show(mosaic, transform=out_trans, ax=ax, title="Merged Treecover2000 (In-Memory)")
plt.show()